In [3]:
import requests
import json
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# ALPHA_VANTAGE_API_KEY

# Fetch stock data from Alpha Vantage API
def fetch_stock_data(symbol, interval='5min'):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={ALPHA_VANTAGE_API_KEY}'
    response = requests.get(url)
    data = response.json()
    return data

symbol = 'MSFT'
interval = '5min'
data = fetch_stock_data(symbol, interval)
data

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'MSFT',
  '3. Last Refreshed': '2023-03-22 20:00:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2023-03-22 20:00:00': {'1. open': '273.2800',
   '2. high': '273.3000',
   '3. low': '273.0000',
   '4. close': '273.0000',
   '5. volume': '3064'},
  '2023-03-22 19:55:00': {'1. open': '273.2800',
   '2. high': '273.2800',
   '3. low': '273.2800',
   '4. close': '273.2800',
   '5. volume': '445'},
  '2023-03-22 19:50:00': {'1. open': '273.0300',
   '2. high': '273.1100',
   '3. low': '273.0300',
   '4. close': '273.1100',
   '5. volume': '1103'},
  '2023-03-22 19:45:00': {'1. open': '273.1500',
   '2. high': '273.2000',
   '3. low': '273.0600',
   '4. close': '273.0600',
   '5. volume': '1028'},
  '2023-03-22 19:40:00': {'1. open': '273.2000',
   '2. high': '273.2000',
   '3. low': '273.2000',
   '4. close': '273.

In [ ]:
# InfluxDB configuration
INFLUXDB_BUCKET = 'your_bucket'
INFLUXDB_ORG = 'your_org'
INFLUXDB_TOKEN = 'your_token'
INFLUXDB_URL = 'http://localhost:8086'

# Store data in InfluxDB
def store_data_in_influxdb(symbol, data):
    client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
    write_api = client.write_api(write_options=SYNCHRONOUS)

    time_series_key = f'Time Series ({data["Meta Data"]["4. Interval"]})'
    for timestamp, values in data[time_series_key].items():
        point = Point("stock_data") \
            .tag("symbol", symbol) \
            .field("open", float(values['1. open'])) \
            .field("high", float(values['2. high'])) \
            .field("low", float(values['3. low'])) \
            .field("close", float(values['4. close'])) \
            .field("volume", int(values['5. volume'])) \
            .time(timestamp, WritePrecision.NS)
        write_api.write(bucket=INFLUXDB_BUCKET, record=[point])

    client.close()

# Example usage
store_data_in_influxdb(symbol, data)
